# AIVLE스쿨 1차 미니프로젝트: 서울시 생활정보 기반 대중교통 수요 분석

<img src = "https://github.com/Jangrae/img/blob/master/bus.png?raw=true" width=800, align="left"/>

- 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
- 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
- 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

---

# [미션 1] 구별 버스정류장 분석

### 로컬 수행(Anaconda)
- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [8]:
# 기본 경로
path = ''

### 구글 콜랩 수행

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [10]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

### 한글 폰트 표시를 위해 설치합니다.

In [12]:
# 한글 표시를 위한 라이브러리 설치
!pip install koreanize_matplotlib -q

### 필요한 라이브러리를 불러옵니다.

In [14]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

In [15]:
# 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정 (예: NanumGothic)
font_path = 'C:\\Windows\\Fonts\\malgun.ttf'   # 폰트 파일 경로
fontprop = fm.FontProperties(fname=font_path, size=12)
plt.rc('font', family=fontprop.get_name())

### 기본 데이터: 1.1 bus_station_boarding_month_202408.csv
- 서울시 버스노선별 정류장별 승하차 인원 정보
- 출처: https://data.seoul.go.kr/dataList/OA-12912/S/1/datasetView.do

### 추가 데이터: 1.1 bus_station_202408.xlsx
- 서울시 버스정류장 위치정보
- 출처: https://data.seoul.go.kr/dataList/OA-15067/S/1/datasetView.do

---

# 1. 데이터 불러오기

- 분석 대상 데이터를 불러와 데이터프레임으로 선언합니다.

### [참고] 데이터 프레임을 불러오고 변수로 저장(CSV 기준으로 진행)

- csv: pd.read_csv('파일이름.csv')
- txt: pd.read_csv('파일이름.csv', sep='구분자')
- xlsx: pd.read_excel('파일이름.xlsx')
- pickle: pd.read_pickle('파일이름.pkl')

## [실습1] 데이터 불러오기

- 대상 파일: 1.1 bus_station_boarding_month_202408.csv
- 불러온 데이터는 bus_station 데이터프레임으로 선언하세요.
- 데이터 불러올 때 참고할 사항
    - 구분자(sep)는 ',' 입니다
    - CP949 인코더를 사용해 주세요
- 가상 정류장은 '버스정류장ARS번호' 열 값이 '~'로 저정되어 있습니다.
- 가상 정류장은 버스정보시스템 상에서 위치 정보를 표시하기 위해 임의로 설정한 것이며, 실제하지 않습니다.
- 그러므로 가상 정류장 행은 삭제합니다.

In [21]:
# 데이터 불러오기
bus_station = pd.read_csv(path + '1.1 bus_station_boarding_month_202408.csv', sep=',', encoding='CP949')

# 가상 정류장 삭제
del_rows = list(bus_station[bus_station['버스정류장ARS번호'] == '~'].index)
bus_station.drop(del_rows, axis=0, inplace=True)

# 확인
bus_station.head()

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20240801,100,100번(하계동~용산구청),01002,창경궁.서울대학교병원(00031),123,134,20240804
1,20240801,7728,7728번(대화동~신촌),19702,DMC두산위브더퍼스트(00038),217,45,20240804
2,20240801,7728,7728번(대화동~신촌),19701,DMC두산위브더퍼스트(00071),22,148,20240804
3,20240801,7728,7728번(대화동~신촌),13259,현대백화점(00056),125,78,20240804
4,20240801,7728,7728번(대화동~신촌),13147,연희동대우아파트(00058),118,33,20240804


In [22]:
# 데이터프레임 크기(행, 열) 확인
num_rows = bus_station.shape[0]
num_columns = bus_station.shape[1]
print(num_rows, num_columns)


1165104 8


---

# 2. 기본 정보 확인 및 전처리

- 분석 대상 테이터를 탐색하고 적절한 전처리를 수행합니다.

## [실습2] 기본 정보 확인

- bus_station 데이터프레임 정보를 확인하세요.
- head(), tail(), info(), describe() 메서드 등을 활용해 보세요.

In [26]:
# head() 메서드로 상위 데이터 확인
bus_station.head()

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20240801,100,100번(하계동~용산구청),01002,창경궁.서울대학교병원(00031),123,134,20240804
1,20240801,7728,7728번(대화동~신촌),19702,DMC두산위브더퍼스트(00038),217,45,20240804
2,20240801,7728,7728번(대화동~신촌),19701,DMC두산위브더퍼스트(00071),22,148,20240804
3,20240801,7728,7728번(대화동~신촌),13259,현대백화점(00056),125,78,20240804
4,20240801,7728,7728번(대화동~신촌),13147,연희동대우아파트(00058),118,33,20240804


In [27]:
# tail() 메서드로 하위 데이터 확인
bus_station.tail()

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
1170229,20240831,3422,3422번(장지공영차고지~한티역),24010,가락시장.가락시장역(00068),29,116,20240903
1170230,20240831,753,753번(구산동~상도동),13002,북가좌동삼거리(00084),73,40,20240903
1170231,20240831,성북13,성북13(동방주택~석계역),08451,동방주택(00028),31,56,20240903
1170232,20240831,3422,3422번(장지공영차고지~한티역),24011,문정로데오거리입구(00025),33,42,20240903
1170233,20240831,107,107번(민락동차고지~종로5가),08269,세아.보광.건영아파트앞(00102),39,108,20240903


In [28]:
# info() 메서드로 열 이름, 데이터 개수, 데이터 형식 등 확인
bus_station.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1165104 entries, 0 to 1170233
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   사용일자        1165104 non-null  int64 
 1   노선번호        1165104 non-null  object
 2   노선명         1165104 non-null  object
 3   버스정류장ARS번호  1165104 non-null  object
 4   역명          1165104 non-null  object
 5   승차총승객수      1165104 non-null  int64 
 6   하차총승객수      1165104 non-null  int64 
 7   등록일자        1165104 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 80.0+ MB


In [29]:
# 표시 형식 변경
pd.set_option('display.float_format', '{:.4f}'.format)

In [30]:
# describe() 메서드로 기술통계정보 확인
bus_station.describe()

,사용일자,승차총승객수,하차총승객수,등록일자
count,1165104.0000,1165104.0000,1165104.0000,1165104.0000
mean,20240816.0110,116.5005,114.2709,20240825.7067
std,8.9479,159.1275,144.4570,26.1626
min,20240801.0000,0.0000,0.0000,20240804.0000
25%,20240808.0000,18.0000,21.0000,20240811.0000
50%,20240816.0000,69.0000,74.0000,20240819.0000
75%,20240824.0000,156.0000,155.0000,20240827.0000
max,20240831.0000,4494.0000,3496.0000,20240903.0000


In [31]:
# 표시 형식 초기화
pd.reset_option('display.float_format')

## [실습3] 버스정류장 위치를 구별로 구분

- '버스정류장ARS번호' 열의 값 앞 2자리가 구를 의미합니다.(예:  '01002'의 앞 2자리 '01')
- '버스정류장ARS번호' 열의 값 앞 2자리를 갖는 '자치구' 열을 추가하세요.
- 참고로, 자치구는 각 정류장이 위치한 구 이름입니다.

In [33]:
# '버스정류장ARS번호' 열 확인
bus_station['버스정류장ARS번호']

0          01002
1          19702
2          19701
3          13259
4          13147
           ...  
1170229    24010
1170230    13002
1170231    08451
1170232    24011
1170233    08269
Name: 버스정류장ARS번호, Length: 1165104, dtype: object

In [34]:
# '자치구' 열 추가
# 참고: df['새열'] = df['기존열'].str[m:n] 형태로 슬라이싱
bus_station['자치구'] = bus_station['버스정류장ARS번호'].str[:2]

# 확인
bus_station['자치구']

0          01
1          19
2          19
3          13
4          13
           ..
1170229    24
1170230    13
1170231    08
1170232    24
1170233    08
Name: 자치구, Length: 1165104, dtype: object

**[참고] 버스정류장 ARS 번호: 01~25까지 앞 숫자 두개가 위치한 구를 의미**

~~~
01. 종로구  02. 중구    03. 용산구  04. 성동구  05. 광진구    06. 동대문구  07. 중랑구
08. 성북구  09. 강북구  10. 도봉구  11. 노원구  12. 은평구    13. 서대문구  14. 마포구
15. 양천구  16. 강서구  17. 구로구  18. 금천구  19. 영등포구  20. 동작구    21. 관악구
22. 서초구  23. 강남구  24. 송파구  25. 강동구
~~~

- 다음 코드를 필히 실행하여 구 코드를 구 이름으로 변경하세요.

In [37]:
# 구 코드를 구 이름으로 변경
bus_station['자치구'] = bus_station['자치구'].map({
    '01': '종로구', '02': '중구',   '03': '용산구', '04': '성동구', '05': '광진구',   '06': '동대문구', '07': '중랑구',
    '08': '성북구', '09': '강북구', '10': '도봉구', '11': '노원구', '12': '은평구',   '13': '서대문구', '14': '마포구',
    '15': '양천구', '16': '강서구', '17': '구로구', '18': '금천구', '19': '영등포구', '20': '동작구',   '21': '관악구',
    '22': '서초구', '23': '강남구', '24': '송파구', '25': '강동구'})

# '버스정류장ARS번호' 열 데이터 형식 변경
bus_station['버스정류장ARS번호'] = bus_station['버스정류장ARS번호'].astype(int)

# 확인
bus_station.head()

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자,자치구
0,20240801,100,100번(하계동~용산구청),1002,창경궁.서울대학교병원(00031),123,134,20240804,종로구
1,20240801,7728,7728번(대화동~신촌),19702,DMC두산위브더퍼스트(00038),217,45,20240804,영등포구
2,20240801,7728,7728번(대화동~신촌),19701,DMC두산위브더퍼스트(00071),22,148,20240804,영등포구
3,20240801,7728,7728번(대화동~신촌),13259,현대백화점(00056),125,78,20240804,서대문구
4,20240801,7728,7728번(대화동~신촌),13147,연희동대우아파트(00058),118,33,20240804,서대문구


## [실습4] 서울지역 데이터 준비

- bus_station 데이터프레임에는 서울 지역 외의 버스정류장이 포함되어 있습니다.
- 예를 들어 다음과 같이 '버스정류장ARS번호' 열 값이 7674 인 행을 검색하면 조회가 됩니다.
- 하지만 네이버 지도에서 7674번 버스 정류장을 검색해 보세요. 이 정류장은 서울 지역이 아닙니다.

In [39]:
# 7674 번 정류장 검색
bus_station.loc[bus_station['버스정류장ARS번호'] == 7674].head()

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자,자치구
190830,20240806,9409,9409번(구미동차고지~신사역),7674,이우중고(00005),1,0,20240809,중랑구
304889,20240809,9409,9409번(구미동차고지~신사역),7674,이우중고(00005),1,0,20240812,중랑구
342468,20240810,9409,9409번(구미동차고지~신사역),7674,이우중고(00005),0,2,20240813,중랑구
417226,20240812,9409,9409번(구미동차고지~신사역),7674,이우중고(00005),2,0,20240815,중랑구
568426,20240816,9409,9409번(구미동차고지~신사역),7674,이우중고(00005),0,1,20240819,중랑구


- 1.1 bus_station_202408.xlsx 엑셀 파일에 서울지역 만의 정류장 정보가 포함되어 있습니다.
- 파일을 읽어와 only_seoul 데이터프레임으로 선언하세요.

In [41]:
# 서울 버스 정류장 데이터 불러오기
# 참고: read_excel() 메서드 사용
only_seoul = pd.read_excel(path +'1.1 bus_station_202408.xlsx')

# 확인
only_seoul.head()

,ARS_ID,정류소명
0,1001,종로2가사거리
1,1002,창경궁.서울대학교병원
2,1003,명륜3가.성대입구
3,1004,종로2가.삼일교
4,1005,혜화동로터리.여운형활동터


- only_seoul 데이터프레임의 'ARS_ID' 열 이름을 '버스정류장ARS번호'로 변경하세요.

In [43]:
# 열 이름 변경: 'ARS_ID ' --> '버스정류장ARS번호'
# 참고: rename() 메서드 사용
only_seoul.rename(columns={'ARS_ID': '버스정류장ARS번호'}, inplace=True)

# 확인
only_seoul.head()


,버스정류장ARS번호,정류소명
0,1001,종로2가사거리
1,1002,창경궁.서울대학교병원
2,1003,명륜3가.성대입구
3,1004,종로2가.삼일교
4,1005,혜화동로터리.여운형활동터


- only_seoul, bus_station 두 데이터프레임을 조인(merge)하세요.
- 조인 결과를 tmp 데이터프레임으로 선언하세요.

In [45]:
# only_seoul, bus_station 데이터프레임 조인
# 참고: '버스정류장ARS번호' 열이 조인 기준, how='inner' 지정
tmp = pd.merge(only_seoul, bus_station, on='버스정류장ARS번호', how='inner')

# 확인
tmp.head()


,버스정류장ARS번호,정류소명,사용일자,노선번호,노선명,역명,승차총승객수,하차총승객수,등록일자,자치구
0,1001,종로2가사거리,20240801,N37,N37번(송파공영차고지~진관공영차고지),종로2가사거리(00032),15,26,20240804,종로구
1,1001,종로2가사거리,20240801,N37,N37번(진관공영차고지~송파공영차고지),종로2가사거리(00089),13,11,20240804,종로구
2,1001,종로2가사거리,20240801,470,470번(상암차고지~안골마을),종로2가사거리(00066),296,288,20240804,종로구
3,1001,종로2가사거리,20240801,741,741번(진관차고지~헌인릉입구),종로2가사거리(00075),259,293,20240804,종로구
4,1001,종로2가사거리,20240802,N37,N37번(송파공영차고지~진관공영차고지),종로2가사거리(00032),12,5,20240805,종로구


---
### <font color="red">[주의] </font> 실습5부터 실습8까지는 tmp 데이터프레임을 대상으로 집계를 수행합니다.
---

## [실습5] 구별 버스정류장 개수 확인

- 자치구별 '버스정류장ARS번호' 개수를 집계하세요.
- 집계 결과를 seoul_bus_station_ars 데이터프레임으로 선언하세요.

In [48]:
# 자치구별 '버스정류장ARS번호' 개수 집계 --> seoul_bus_station_ars 데이터프레임으로 선언
# 참고: groupby() 메서드 사용, as_index=False 지정, nunique() 집계
seoul_bus_station_ars = tmp.groupby('자치구', as_index=False)['버스정류장ARS번호'].nunique()

# 확인
seoul_bus_station_ars.head()


,자치구,버스정류장ARS번호
0,강남구,502
1,강동구,376
2,강북구,410
3,강서구,565
4,관악구,465


- '버스정류장ARS번호' 열 이름을 '정류장수' 로 변경하세요.

In [50]:
# 열 이름 변경: '버스정류장ARS번호' --> '정류장수'
# 참고: rename() 메서드 사용
seoul_bus_station_ars.rename(columns={'버스정류장ARS번호': '정류장수'}, inplace=True)

# 확인
seoul_bus_station_ars.head()

,자치구,정류장수
0,강남구,502
1,강동구,376
2,강북구,410
3,강서구,565
4,관악구,465


## [실습6] 구별 버스노선 개수 확인

- 자치구별 '노선번호' 개수를 집계하세요.
- 집계 결과를 seoul_bus_station_line 데이터프레임으로 선언하세요.

In [52]:
# 자치구별 '노선번호' 개수 집계 --> seoul_bus_station_line 데이터프레임으로 선언
# 참고: groupby() 메서드 사용, as_index=False 지정, nunique() 집계
seoul_bus_station_line = tmp.groupby('자치구', as_index=False)['노선번호'].nunique()

# 확인
seoul_bus_station_line.tail(10)

,자치구,노선번호
15,성동구,58
16,성북구,100
17,송파구,62
18,양천구,57
19,영등포구,101
20,용산구,70
21,은평구,76
22,종로구,107
23,중구,102
24,중랑구,50


- '노선번호' 열 이름을 '노선수'로 변경하세요.

In [54]:
# 열 이름 변경: '노선번호' --> '노선수'
# 참고: rename() 메서드 사용
seoul_bus_station_line.rename(columns={'노선번호': '노선수'}, inplace=True)

# 확인
seoul_bus_station_line.head(10)

,자치구,노선수
0,강남구,99
1,강동구,24
2,강북구,71
3,강서구,54
4,관악구,82
5,광진구,43
6,구로구,81
7,금천구,56
8,노원구,61
9,도봉구,57


## [실습7] 구별 승차 총 승객수, 하차 총 승객수 구하기

- 자치구별 '승차총승객수', '하차총승객수' 합을 집계하세요.
- 집계 결과를 seoul_bus_station_sum 데이터프레임으로 선언하세요.

In [56]:
# 자치구별 '승차총승객수', '하차총승객수' 합 집계 --> seoul_bus_station_sum 데이터프레임으로 선언
# 참고: groupby() 메서드 사용, as_index=False 지정, sum() 집계
seoul_bus_station_sum = tmp.groupby('자치구', as_index=False)[['승차총승객수', '하차총승객수']].sum()

# 확인
seoul_bus_station_sum.head(10)


,자치구,승차총승객수,하차총승객수
0,강남구,9113021,8640353
1,강동구,3306588,3242901
2,강북구,5244436,5077041
3,강서구,5244615,5218001
4,관악구,7691243,7850254
5,광진구,3207725,3228754
6,구로구,5525646,5324566
7,금천구,3990114,3839246
8,노원구,4609496,4535591
9,도봉구,3575712,3493061


## [실습8] 구별 승차 평균 승객수, 하차 평균 승객수 구하기

- 자치구별 '승차총승객수', '하차총승객수' 평균을 집계하세요.
- 집계 결과를 seoul_bus_station_mean 데이터프레임으로 선언하세요.

In [58]:
# 자치구별 '승차총승객수', '하차총승객수' 평균 집계 --> seoul_bus_station_mean 데이터프레임으로 선언
# 참고: groupby() 메서드 사용, as_index=False 지정, mean() 집계
seoul_bus_station_mean = tmp.groupby('자치구', as_index=False)[['승차총승객수', '하차총승객수']].mean()

# 확인
seoul_bus_station_mean.head()


,자치구,승차총승객수,하차총승객수
0,강남구,135.163908,128.153318
1,강동구,94.638885,92.816079
2,강북구,138.244306,133.831743
3,강서구,93.106837,92.634362
4,관악구,147.777793,150.832994


- 다음과 같이 seoul_bus_station_mean 데이터프레임의 열 이름을 변경하세요.
    - '승차총승객수' --> '승차평균승객수', '하차총승객수' --> '하차평균승객수'

In [60]:
# 열 이름 변경: '승차총승객수' --> '승차평균승객수', '하차총승객수' --> '하차평균승객수'
# 참고: rename() 메서드 사용
seoul_bus_station_mean.rename(columns={'승차총승객수': '승차평균승객수'}, inplace=True)
seoul_bus_station_mean.rename(columns={'하차총승객수': '하차평균승객수'}, inplace=True)

# 확인
seoul_bus_station_mean.head()

,자치구,승차평균승객수,하차평균승객수
0,강남구,135.163908,128.153318
1,강동구,94.638885,92.816079
2,강북구,138.244306,133.831743
3,강서구,93.106837,92.634362
4,관악구,147.777793,150.832994


## [실습9] 데이터 프레임 합치기

- 위 과정에서 만들어진 다음 4개 데이터프레임을 조인(merge)하세요.
    - seoul_bus_station_ars
    - seoul_bus_station_line
    - seoul_bus_station_sum
    - seoul_bus_station_mean
- 조인 결과를 seoul_bus_station 데이터프레임으로 선언하세요.

In [62]:
# 4개 데이터프레임 조인
# 참고: pd.merge() 함수를 3번 사용해야 함
# 참고: '자치구' 열이 조인 기준, how='inner' 지정
bus = pd.merge(seoul_bus_station_ars, seoul_bus_station_line, how='inner', on='자치구')
bus = pd.merge(bus, seoul_bus_station_sum, how='inner', on='자치구')
seoul_bus_station = pd.merge(bus, seoul_bus_station_mean, how='inner', on='자치구')

# 확인
seoul_bus_station.head()

,자치구,정류장수,노선수,승차총승객수,하차총승객수,승차평균승객수,하차평균승객수
0,강남구,502,99,9113021,8640353,135.163908,128.153318
1,강동구,376,24,3306588,3242901,94.638885,92.816079
2,강북구,410,71,5244436,5077041,138.244306,133.831743
3,강서구,565,54,5244615,5218001,93.106837,92.634362
4,관악구,465,82,7691243,7850254,147.777793,150.832994


- 다음과 같이 seoul_bus_station 데이터프레임의 열 이름을 변경하세요
    - '버스정류장ARS번호' --> '정류장수', '노선번호' --> '노선수'

In [64]:
# 열 이름 변경: '버스정류장ARS번호' --> '정류장수', '노선번호' --> '노선수'
# 참고: rename() 메서드 사용
seoul_bus_station.rename(columns={'버스정류장ARS번호': '정류장수'}, inplace=True)
seoul_bus_station.rename(columns={'노선번호': '노선수'}, inplace=True)

# 롹인
seoul_bus_station.head()

,자치구,정류장수,노선수,승차총승객수,하차총승객수,승차평균승객수,하차평균승객수
0,강남구,502,99,9113021,8640353,135.163908,128.153318
1,강동구,376,24,3306588,3242901,94.638885,92.816079
2,강북구,410,71,5244436,5077041,138.244306,133.831743
3,강서구,565,54,5244615,5218001,93.106837,92.634362
4,관악구,465,82,7691243,7850254,147.777793,150.832994


- seoul_bus_station 데이터프레임을 df_seoul_bus_station.csv 파일로 저장하세요.

In [66]:
# csv 파일로 데이터프레임 저장 (파일명: df_seoul_bus_station.csv)
# 참고: to_csv() 메서드를 사용, index=False 지정
seoul_bus_station.to_csv(path + 'df_seoul_bus_station.csv', index=False)

In [67]:
# csv 파일 확인
df_seoul_bus_station = pd.read_csv(path + 'df_seoul_bus_station.csv')
df_seoul_bus_station

,자치구,정류장수,노선수,승차총승객수,하차총승객수,승차평균승객수,하차평균승객수
0,강남구,502,99,9113021,8640353,135.163908,128.153318
1,강동구,376,24,3306588,3242901,94.638885,92.816079
2,강북구,410,71,5244436,5077041,138.244306,133.831743
3,강서구,565,54,5244615,5218001,93.106837,92.634362
4,관악구,465,82,7691243,7850254,147.777793,150.832994
5,광진구,278,43,3207725,3228754,118.756247,119.534782
6,구로구,478,81,5525646,5324566,125.119353,120.566221
7,금천구,344,56,3990114,3839246,129.954208,125.040581
8,노원구,495,61,4609496,4535591,88.736303,87.313576
9,도봉구,366,57,3575712,3493061,110.579911,108.023905


---

In [69]:
print(df_seoul_bus_station.dtypes)

자치구         object
정류장수         int64
노선수          int64
승차총승객수       int64
하차총승객수       int64
승차평균승객수    float64
하차평균승객수    float64
dtype: object


# 3. 데이터 분석

- 다양한 분석 과정을 통해 인사이트를 도출하세요.

## [실습10] 시각화 분석

- 다양한 열를 기준으로 그래프를 그려보고 인사이트를 도출해보세요.

In [72]:
# 주석
# 막대그래프
# 데이터 준비
df_melted = df_seoul_bus_station.melt(id_vars='자치구', value_vars=['정류장수', '노선수'], 
                                        var_name='종류', value_name='수')

# 그래프 그리기
plt.figure(figsize=(12, 6))
sns.barplot(data=df_melted, x='자치구', y='수', hue='종류')
plt.title('자치구별 정류장수 및 노선수')
plt.xticks(rotation=45)
plt.ylabel('수')
plt.legend(title='종류')
plt.tight_layout()
plt.show()



NameError: name 'sns' is not defined

<Figure size 1200x600 with 0 Axes>

In [ ]:
# 서브플롯 생성
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# 첫 번째 산점도: 정류장수 vs 승차총승객수
sns.scatterplot(x='정류장수', y='승차총승객수', hue='자치구', data=df_seoul_bus_station, ax=axes[0])
axes[0].set_title('정류장수와 승차총승객수의 관계')

# 두 번째 산점도: 노선수 vs 승차총승객수
sns.scatterplot(x='노선수', y='승차총승객수', hue='자치구', data=df_seoul_bus_station, ax=axes[1])
axes[1].set_title('노선수와 승차총승객수의 관계')

# 레이아웃 조정
plt.tight_layout()
plt.show()

In [ ]:
# 주석
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 시각화
sns.pairplot(df_seoul_bus_station)
plt.show()



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# '자치구' 컬럼을 제외하고 상관관계 행렬 계산
correlation_matrix = df_seoul_bus_station.drop(columns=['자치구']).corr()

# 히트맵 그리기
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', 
            xticklabels=correlation_matrix.columns, yticklabels=correlation_matrix.columns)
plt.title('변수 간 상관관계 히트맵')
plt.show()

* 히트맵을 통한 인사이트
1. 정류장수와 승차총승객수 > 상관계수: 0.5 <br>8
정류장수가 많을수록 승차총승객수가 증가하는 경향이 있습니다. 이는 정류장이 많은 지역일수록 대중교통 이용이 활발하다는 것을 시사합니다.
2. 정류장수와 하차총승객수 > 상관계수: 0 <br>.59
정류장수가 많을수록 하차총승객수도 증가하는 경향이 있습니다. 이는 정류장이 많은 지역이 승차뿐만 아니라 하차에서도 이용량이 많음을 나타냅니다.
3. 노선수와 승차총승객수 > 상관계수: <br> 0.73
노선수가 많을수록 승차총승객수가 높아지는 경향이 강합니다. 이는 다양한 노선이 존재할수록 승객들이 더 많이 이용하게 된다는 것을 의미합니다.
4. 노선수와 하차총승객수 > 상관계 <br>수: 0.72
노선수가 많을수록 하차총승객수도 증가합니다. 이는 노선의 다양성이 승객의 이동을 촉진한다는 것 5을 나타냅니다.
6. 승차평균승객수와 다른 변수들 > 관계수: -0.23 (정류수와), - <br>0.20 (노선수와)
승차평균승객수는 다른 변수들과 부의 상관관계를 보입니다. 이는 평균 승객 수가 높지 않을수록 정류장수나 노선수가 많아도 승차량이 적게 나타날 수6있음을 의미합니다.
7. 하차평균승객수와 다른 변수들 > 상관계수: -0.20 (정류장수와), <br> -0.20 (노선수와)
하차평균승객수도 비슷한 경향을 보입니다. 이는 특정 지역의 평균 승객 수가 낮을 경우, 정류장이나 노선의 수와 관계없이 하차량이 적게 나타날 <br>수 있음을 시사합니다. 

* 결론
정류장수와 노선수는 승차총승객수 및 하차총승객수와 양의 상관관계를 가지며, 이는 대중교통의 이용량에 긍정적인 영향을 미친다는 것을 나타냅니다.
평균 승객 수는 다른 변수들과 부의 상관관계를 보여, 특정 지역의 대중교통 이용 패턴을 보다 깊이 이해할 필요가 있습니다. 평균 승객 수가 낮은 지역은 대중교통 이용을 촉진하기 위한 추가 등의 전략을 모색할 수 있습니다.